In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Fare split: 

1.   80 - 20 train - test(e poi da 80 levi 10% (di 100) random) con stratified (cioè che ognuno dei tre gruppi ha stessa percentuale di etichette)
2.   di nuovo stratified ma invece delle etichette, il sampling è fatto in base al partito
3. un ultimo in cui i partiti politici che fan parte del training non stanno nel test

In questo file viene fatta solo la 1

In [ ]:
pip install scikit-learn

In [ ]:
 import numpy as np
 import json
 from sklearn.model_selection import StratifiedKFold
 from sklearn.model_selection import train_test_split

In [ ]:
ds_file = "/content/gdrive/MyDrive/Colab Notebooks/fakenews_data/github/join_d1.json"

In [ ]:
apiContent = open(ds_file)
data = json.load(apiContent)
apiContent.close()

In [ ]:
X = []
Y = []

for art in data:
  Y.append(art["verdict"])
  X.append(art["statement"])

In [ ]:
print(len(X))
print(len(Y))
print(len(data))

3577
3577
3577


In [ ]:
v = 0
f = 0
n = 0

for art in data:
  if art["verdict"] == "Vero":
    v += 1
  elif art["verdict"] == "Falso":
    f += 1
  elif art["verdict"] == "Nì":
    n += 1

print(v)
print(n)
print(f)
print(len(data)==(v+f+n))

1255
1512
810
True


lievemente biased verso il True, poche per il False

procediamo con svm, tf-idf per investigare bias negli statements

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(X)
df = pd.DataFrame(tfIdf[1].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(25))

# the TF-IDF value increases with every appearance of the word in a document, but is gradually decreased with every appearance in other documents
# define how important a word is for a document

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2))
features = tfidf.fit_transform(X).toarray()
labels = Y
features.shape

(3577, 4284)

 ognuno dei 3577 verdetti è rapresentato da 4284 features, che rappresentano il tf idf score di unigrammi e bigrammi

In [ ]:
#Ensure that the labels are encoded as numerical values, such as integers or one-hot encoding
def numerazione(Y):
  Y1 = []
  for esito in Y:
    if esito == "Vero":
      Y1.append(0)
    elif esito == "Falso":
      Y1.append(2)
    elif esito == "Nì":
      Y1.append(1)
  return Y1

Y1 = numerazione(Y)

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np

In [ ]:
chi2_stat, p_values = chi2(features, Y1)

In [ ]:
feature_names = tfidf.get_feature_names_out()
correlated_terms = [(feature_names[i], chi2_stat[i]) for i in range(len(feature_names))]
correlated_terms.sort(key=lambda x: x[1], reverse=True)

In [ ]:
label_values = [0,1,2]
label_names = ["Vero", "Nì", "Falso"]
N = 10
for label in label_values:

  features_chi2 = chi2(features, np.logical_and(Y1,label))
  indices = np.argsort(features_chi2[0])

  feature_names = np.array(tfidf.get_feature_names_out())[indices]

  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  
  print("# '{}':".format(label_names[label]))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

parte 2

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2))
features = tfidf.fit_transform(X).toarray()
features.shape

(3577, 4284)

In [ ]:
#Ensure that the labels are encoded as numerical values, such as integers or one-hot encoding
Y1 = []
for esito in Y:
  if esito == "Vero":
    Y1.append(0)
  elif esito == "Falso":
    Y1.append(2)
  elif esito == "Nì":
    Y1.append(1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features,Y1, test_size=0.1, train_size=0.9, random_state=7, shuffle=True, stratify=Y1)
# Y1 : lista dei verdetti in formato int

In [ ]:
#from imblearn.over_sampling import RandomOverSampler
#X_train, y_train = RandomOverSampler(random_state=123).fit_resample(X_train, y_train)
#potrebbe risultare utile per bilanciare i falsi che sono pochi

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(random_state=123, kernel='linear') # Linear Kernel

In [ ]:
clf.fit(X_train,y_train)

SVC(kernel='linear', random_state=123)

In [ ]:
print("Accuracy score {:.2f} %\n".format(clf.score(X_test,y_test)*100))

Accuracy score 40.22 %



**FASE 2.5: AGGIUNGENDO IL POLITICO CAMBIA QUALCOSA?**

In [ ]:
X1 = []
X2 = []
Y = []

for art in data:
  Y.append(art["verdict"])
  X1.append(art["statement"])
  X2.append(art["politician"])
#Ensure that the labels are encoded as numerical values, such as integers or one-hot encoding
Y1 = []
for esito in Y:
  if esito == "Vero":
    Y1.append(0)
  elif esito == "Falso":
    Y1.append(2)
  elif esito == "Nì":
    Y1.append(1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, ngram_range=(1, 2))
X1 = tfidf.fit_transform(X1)
tfidf2 = TfidfVectorizer(sublinear_tf=True, min_df=1, ngram_range=(2, 3))
X2 = tfidf2.fit_transform(X2)

df = pd.DataFrame({'A': [X1], 'B': [X2]})

In [ ]:
X1.shape

(3577, 61832)

In [ ]:
X2.shape

(3577, 369)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df,Y1, test_size=0.1, train_size=0.9, random_state=7, shuffle=True, stratify=Y1)

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(random_state=123, kernel='linear') # Linear Kernel

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
print("Accuracy score {:.2f} %\n".format(clf.score(X_test,y_test)*100))

**FASE 3:  RANKING TASK**

dato il claim noi abbiamo una quantità di testi associata che spiega perchè è vero o falso. questo testo (content) è rilevante per questo claim. l'algoritmo deve imparare a riconoscere data una frase quale contenuto ampio è il più simile.
test:
input: lista di claim, lista di spiegazioni
associare claim a spiegazione
valutazione: recall impossibile da stabilire perchè la recall è il mondo esterno. recall alta = sovrageneralizzazione, la recall perfetta è 1.
per i task di ranking si guarda la 'mean average precision a x' : ranking di tutti i testi e poi verifichi se il documento corretto è nei primi x testi.

imbellimento: tool di generazione che cambia il claim in modo molto simile ma non esattamente la stessa


In [ ]:
apiContent = open(ds_file)
data = json.load(apiContent)
apiContent.close()

In [ ]:
X = []
Y = []
Y_frasi = []

# X -> statements
# Y -> contents

for art in data:
  X.append(art["statement"])
  Y_frasi.append(art["content"])
  Y.append(' '.join(art["content"]))

print(len(Y))

3577


In [ ]:
Y

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, ngram_range=(1, 2))
X_tfidf = tfidf.fit_transform(X).toarray()

In [ ]:
Y_tfidf = tfidf.fit_transform(Y).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf,Y_tfidf, test_size=0.1, train_size=0.9, random_state=7, shuffle=True, stratify=Y_tfidf)

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(random_state=123, kernel='linear') # Linear Kernel

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
print("Accuracy score {:.2f} %\n".format(clf.score(X_test,y_test)*100))